**U1 (Target Use Case):**   
One could use this dataset to investigate whether different neighborhoods (indicated by zip code or geographic regions) have distinct food safety patterns, such as passing rate, complaint rate, risks, violations, and violation types, and further stratify the data by other factors (such as facility types and time range).

In [1]:
import pandas as pd
import numpy as np
import re
import string

In [2]:
input_data = pd.read_csv("data/open_refine_processed/Food-Inspections-openrefine.csv")
input_data.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Facility Type Cleaned,Risk,Address,City,State,Zip,Inspection Date,Inspection Date Transformed,Inspection Type,Inspection Type Cleaned,Results,Violations,Latitude,Longitude,Location
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,Restaurant,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,08/28/2017,2017-08-28,License,license,Pass,NaN,41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
1,2079129,JET'S PIZZA,JET'S PIZZA,2522268.0,Restaurant,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607.0,08/28/2017,2017-08-28,License,license,Not Ready,NaN,41.881572,-87.653052,"(41.88157249576794, -87.65305233593274)"
2,2079125,ROOM 1520,ROOM 1520,2446638.0,Special Event,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607.0,08/28/2017,2017-08-28,License Re-Inspection,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328,"(41.88657652150854, -87.66532812140231)"
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,Restaurant,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,08/28/2017,2017-08-28,License,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,CHARTER SCHOOL,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639.0,08/28/2017,2017-08-28,License Re-Inspection,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711,"(41.921675488910864, -87.7767113569357)"


In [3]:
input_data = input_data.drop(columns=['Facility Type', 'Inspection Date', 'Inspection Type']).rename(columns={
    'Facility Type Cleaned': 'Facility Type', 
    'Inspection Date Transformed': 'Inspection Date', 
    'Inspection Type Cleaned': 'Inspection Type'
})

In [4]:
input_data = input_data.rename(columns={
    'Inspection ID': 'inspection_id',
    'DBA Name': 'name_dba',
    'AKA Name': 'name_aka',
    'License #': 'license_id',
    'Facility Type': 'facility_type', 
    'Risk': 'risk',
    'Address': 'address',
    'City': 'city',
    'State': 'state',
    'Zip': 'zip',
    'Inspection Date': 'inspection_date', 
    'Inspection Type': 'inspection_type',
    'Results': 'results',
    'Violations': 'violations',
    'Latitude': 'latitude',
    'Longitude': 'longitude'
})
input_data.head()

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,Location
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,2017-08-28,license,Pass,NaN,41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
1,2079129,JET'S PIZZA,JET'S PIZZA,2522268.0,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607.0,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052,"(41.88157249576794, -87.65305233593274)"
2,2079125,ROOM 1520,ROOM 1520,2446638.0,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607.0,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328,"(41.88657652150854, -87.66532812140231)"
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639.0,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711,"(41.921675488910864, -87.7767113569357)"


# 1. Basic data exploration and cleaning

## 1.1 data types and duplicated column(s)

**[exploration]**   
Check data types and convert data types:

In [5]:
input_data.dtypes

inspection_id        int64
name_dba            object
name_aka            object
license_id         float64
facility_type       object
risk                object
address             object
city                object
state               object
zip                float64
inspection_date     object
inspection_type     object
results             object
violations          object
latitude           float64
longitude          float64
Location            object
dtype: object

In [6]:
input_data['inspection_id'] = input_data['inspection_id'].astype(str)
input_data['license_id'] = input_data['license_id'].astype(str)
input_data['license_id'] = input_data['license_id'].str.replace('.0', '', regex=False)
input_data['zip'] = input_data['zip'].astype(str).str.replace('.0', '', regex=False)
input_data['inspection_date'] = pd.to_datetime(input_data['inspection_date'])
input_data.dtypes

inspection_id              object
name_dba                   object
name_aka                   object
license_id                 object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
violations                 object
latitude                  float64
longitude                 float64
Location                   object
dtype: object

**[exploration]**   
Check missing values:

In [7]:
input_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153810 entries, 0 to 153809
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   inspection_id    153810 non-null  object        
 1   name_dba         153810 non-null  object        
 2   name_aka         151267 non-null  object        
 3   license_id       153810 non-null  object        
 4   facility_type    149250 non-null  object        
 5   risk             153744 non-null  object        
 6   address          153810 non-null  object        
 7   city             153651 non-null  object        
 8   state            153802 non-null  object        
 9   zip              153810 non-null  object        
 10  inspection_date  153810 non-null  datetime64[ns]
 11  inspection_type  153809 non-null  object        
 12  results          153810 non-null  object        
 13  violations       123012 non-null  object        
 14  latitude         153

In [8]:
input_data.isnull().sum()

inspection_id          0
name_dba               0
name_aka            2543
license_id             0
facility_type       4560
risk                  66
address                0
city                 159
state                  8
zip                    0
inspection_date        0
inspection_type        1
results                0
violations         30798
latitude             544
longitude            544
Location             544
dtype: int64

**[exploration]**  
Check whether the "location" column has the same information with the "latitude" and "longitude" columns:

In [9]:
def check_location(x):
    result = True
    e = 10e-5
    if (not pd.isnull(x['Location'])) and len(x['Location']) > 0:
        la, lo = x['Location'].strip('(|)').split(',')
        la, lo = float(la), float(lo)
        result = (abs(la - x['latitude']) < e) and (abs(lo - x['longitude']) < e)
    return result
    
sum(input_data.apply(check_location, axis=1)) == len(input_data)

True

**[cleaning]**  
So let's remove the "location" column since it does not provide any additional information:

In [10]:
input_data.drop(columns='Location', inplace=True)

In [11]:
input_data.head()

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JET'S PIZZA,JET'S PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711


## 1.2 string cleanup for 'name_dba', 'name_aka', 'address', 'city', 'zip', 'state'

**[cleaning]**  
Clean up strings:

In [12]:
def clean_strings(x):
    return x.upper().replace('.', '').replace(',', '').replace("'", '').strip() if (not pd.isnull(x)) else x

for column in ['name_dba', 'name_aka', 'address', 'city']:
    input_data[column] = input_data[column].apply(clean_strings)

input_data.head()

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JETS PIZZA,JETS PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711


**[exploration]**  
Check whether the "city", and "state" columns look good:

In [13]:
input_data['city'].value_counts()

CHICAGO                 153435
CCHICAGO                    39
SCHAUMBURG                  18
MAYWOOD                     15
ELK GROVE VILLAGE           12
CHESTNUT STREET              8
INACTIVE                     8
SKOKIE                       8
EVANSTON                     7
CICERO                       6
CHICAGOCHICAGO               6
CHCHICAGO                    6
CHARLES A HAYES              6
ELMHURST                     5
WORTH                        5
ALSIP                        4
SUMMIT                       4
CALUMET CITY                 4
OAK PARK                     4
EAST HAZEL CREST             3
NILES NILES                  3
CHICAGOI                     3
SCHILLER PARK                3
CHCICAGO                     3
BEDFORD PARK                 2
BANNOCKBURNDEERFIELD         2
BERWYN                       2
NAPERVILLE                   2
STREAMWOOD                   2
CHICAGO HEIGHTS              2
BLUE ISLAND                  2
312CHICAGO                   2
OLYMPIA 

**[cleaning]**  
We do notice some misspelling of "CHICAGO", as well as "INACTIVE",
let's correct them:

In [14]:
input_data['city'] = input_data['city'].apply(
    lambda x: 'CHICAGO' if x in ['CCHICAGO', 'CHICAGOCHICAGO', 'CHCHICAGO', 'CHICAGOI', 'CHCICAGO', '312CHICAGO'] else (np.nan if x=='INACTIVE' else x))
input_data['city'].value_counts()

CHICAGO                 153494
SCHAUMBURG                  18
MAYWOOD                     15
ELK GROVE VILLAGE           12
SKOKIE                       8
CHESTNUT STREET              8
EVANSTON                     7
CHARLES A HAYES              6
CICERO                       6
ELMHURST                     5
WORTH                        5
ALSIP                        4
OAK PARK                     4
CALUMET CITY                 4
SUMMIT                       4
EAST HAZEL CREST             3
SCHILLER PARK                3
NILES NILES                  3
STREAMWOOD                   2
BLUE ISLAND                  2
CHICAGO HEIGHTS              2
NAPERVILLE                   2
BANNOCKBURNDEERFIELD         2
BEDFORD PARK                 2
BERWYN                       2
GLENCOE                      1
FRANKFORT                    1
DES PLAINES                  1
OAK LAWN                     1
OLYMPIA FIELDS               1
BROADVIEW                    1
LAKE ZURICH                  1
WESTMONT

**[exploration]**  
Check whether the "zip" column has valid values:  
There are also some missing values "nan". We can also check if all zip codes are valid (5 digits). 

In [15]:
sorted(list(set(input_data['zip'])))

['60007',
 '60015',
 '60018',
 '60022',
 '60044',
 '60047',
 '60076',
 '60077',
 '60107',
 '60108',
 '60126',
 '60148',
 '60153',
 '60155',
 '60176',
 '60193',
 '60201',
 '60202',
 '60302',
 '60402',
 '60406',
 '60409',
 '60411',
 '60423',
 '60429',
 '60440',
 '60453',
 '60458',
 '60461',
 '60477',
 '60478',
 '60482',
 '60501',
 '60540',
 '60559',
 '60601',
 '60602',
 '60603',
 '60604',
 '60605',
 '60606',
 '60607',
 '60608',
 '60609',
 '60610',
 '60611',
 '60612',
 '60613',
 '60614',
 '60615',
 '60616',
 '60617',
 '60618',
 '60619',
 '60620',
 '60621',
 '60622',
 '60623',
 '60624',
 '60625',
 '60626',
 '60627',
 '60628',
 '60629',
 '60630',
 '60631',
 '60632',
 '60633',
 '60634',
 '60636',
 '60637',
 '60638',
 '60639',
 '60640',
 '60641',
 '60642',
 '60643',
 '60644',
 '60645',
 '60646',
 '60647',
 '60649',
 '60651',
 '60652',
 '60653',
 '60654',
 '60655',
 '60656',
 '60657',
 '60659',
 '60660',
 '60661',
 '60666',
 '60706',
 '60707',
 '60714',
 '60803',
 '60804',
 '60805',
 '60827',


In [16]:
input_data[input_data['zip']=='nan'].head()

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
5927,2028349,KITCHFIX,KITCHFIX GOLD COAST,2418085,restaurant,Risk 2 (Medium),1165 N STATE ST,NaN,IL,nan,2017-04-14,complaint,Pass,42. APPROPRIATE METHOD OF HANDLING OF FOOD (IC...,41.903769,-87.628532
7750,1995776,MONICAS PIZZA,MONICAS PIZZA,2522031,restaurant,Risk 1 (High),6446-6448 S PULASKI RD,NaN,IL,nan,2017-03-17,license,Pass,NaN,41.775402,-87.722822
8422,1990124,7-ELEVEN #37622B,7-ELEVEN #37622B,2492548,grocery,Risk 2 (Medium),4 W JACKSON BLVD,NaN,IL,nan,2017-03-07,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.878286,-87.627804
8483,1990130,7-ELEVEN #37622B,7-ELEVEN #37622B,2492551,grocery,Risk 2 (Medium),4 W JACKSON BLVD,NaN,IL,nan,2017-03-07,license,Pass,NaN,41.878286,-87.627804
8491,1990118,7-ELEVEN # 37622A,7 ELEVEN,2442933,grocery,Risk 1 (High),4 W JACKSON BLVD,NaN,IL,nan,2017-03-07,canvass,Out of Business,NaN,41.878286,-87.627804


Except for the 'nan' values, all other values have 5 digits:

In [17]:
input_data[(input_data['zip']!='nan') & (input_data['zip'].str.len()!= 5)]

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude


**[cleaning]**  
Convert all 'nan' zip values to be null:

In [18]:
input_data['zip'] = input_data['zip'].apply(lambda x: np.nan if x=='nan' else x)
input_data

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JETS PIZZA,JETS PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153805,67732,WOLCOTTS,TROQUET,1992039,restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.961606,-87.675967
153806,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601,2010-01-04,tag removal,Pass,NaN,41.884586,-87.631010
153807,52234,CAFE 608,CAFE 608,2013328,restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,license re-inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.938007,-87.644755
153808,70269,MRDANIELS,MRDANIELS,1899292,restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.938443,-87.768318


Confirm the change:

In [19]:
list(set(input_data['zip']))

[nan,
 '60461',
 '60660',
 '60047',
 '60107',
 '60629',
 '60706',
 '60076',
 '60640',
 '60176',
 '60478',
 '60643',
 '60645',
 '60429',
 '60615',
 '60646',
 '60638',
 '60627',
 '60653',
 '60661',
 '60477',
 '60625',
 '60559',
 '60644',
 '60651',
 '60614',
 '60827',
 '60440',
 '60007',
 '60621',
 '60620',
 '60402',
 '60637',
 '60155',
 '60406',
 '60619',
 '60634',
 '60126',
 '60652',
 '60616',
 '60630',
 '60202',
 '60603',
 '60618',
 '60602',
 '60707',
 '60453',
 '60617',
 '60609',
 '60631',
 '60482',
 '60639',
 '60655',
 '60641',
 '60015',
 '60654',
 '60657',
 '60077',
 '60201',
 '60656',
 '60803',
 '60108',
 '60423',
 '60649',
 '60804',
 '60302',
 '60647',
 '60612',
 '60458',
 '60605',
 '60604',
 '60636',
 '60501',
 '60022',
 '60659',
 '60193',
 '60622',
 '60411',
 '60642',
 '60610',
 '60018',
 '60148',
 '60666',
 '60606',
 '60044',
 '60714',
 '60601',
 '60540',
 '60626',
 '60632',
 '60628',
 '60153',
 '60613',
 '60607',
 '60805',
 '60611',
 '60623',
 '60624',
 '60409',
 '60633',
 '60

**[exploaration]**  
Check the state column:  
There are only 8 records with missing "state" while all others are "IL". We can also see that we might be able to infer the "state" from "zip". We will take care of this later.

In [20]:
list(set(input_data['state']))

[nan, 'IL']

In [21]:
input_data['state'].isnull().sum()

8

In [22]:
input_data[input_data['state'].isnull()]

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
20927,1950389,AMY BECK CAKE DESIGN LLC,AMY BECK CAKE DESIGN LLC,2079264,bakery,Risk 1 (High),636 N RACINE AVE,NaN,NaN,60642,2016-08-15,canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.893380,-87.657588
43620,1566316,AMY BECK CAKE DESIGN LLC,AMY BECK CAKE DESIGN LLC,2079264,bakery,Risk 1 (High),636 N RACINE AVE,NaN,NaN,60642,2015-08-17,canvass,Pass,21. * CERTIFIED FOOD MANAGER ON SITE WHEN POTE...,41.893380,-87.657588
44075,1562023,AMY BECK CAKE DESIGN LLC,AMY BECK CAKE DESIGN LLC,2079264,bakery,Risk 1 (High),636 N RACINE AVE,NaN,NaN,60642,2015-08-10,canvass,Fail,21. * CERTIFIED FOOD MANAGER ON SITE WHEN POTE...,41.893380,-87.657588
70837,1459452,AMY BECK CAKE DESIGN LLC,AMY BECK CAKE DESIGN LLC,2079264,bakery,Risk 1 (High),636 N RACINE AVE,NaN,NaN,60642,2014-05-06,canvass,Pass,42. APPROPRIATE METHOD OF HANDLING OF FOOD (IC...,41.893380,-87.657588
88043,1353435,EMPORIUM ARCADE BAR,EMPORIUM ARCADE BAR,2133873,tavern,Risk 3 (Low),1366 N MILWAUKEE AVE,CHICAGO,NaN,60642,2013-07-02,canvass,Out of Business,NaN,41.906543,-87.671693
108931,1229551,EMPORIUM ARCADE BAR,EMPORIUM ARCADE BAR,2133873,tavern,Risk 3 (Low),1366 N MILWAUKEE AVE,CHICAGO,NaN,60642,2012-06-11,license re-inspection,Pass,NaN,41.906543,-87.671693
109306,1229384,EMPORIUM ARCADE BAR,EMPORIUM ARCADE BAR,2133873,tavern,Risk 3 (Low),1366 N MILWAUKEE AVE,CHICAGO,NaN,60642,2012-06-05,license,Fail,12. HAND WASHING FACILITIES: WITH SOAP AND SAN...,41.906543,-87.671693
133613,579263,AMY BECK CAKE DESIGN LLC,AMY BECK CAKE DESIGN LLC,2079264,bakery,Risk 1 (High),636 N RACINE AVE,NaN,NaN,60642,2011-02-22,license,Pass,NaN,41.893380,-87.657588


In [23]:
input_data[input_data['zip']=='60642'][['zip', 'state', 'city']].drop_duplicates()

,zip,state,city
174,60642,IL,CHICAGO
20927,60642,NaN,NaN
88043,60642,NaN,CHICAGO


## 1.3 Clean up address variations and incorrect/missing city/zip/state

**[exploaration]**  
Further check if there are different variations of addresses for the same location (latitude & longitude) and city/state/zip:

In [24]:
temp = input_data[['address', 'latitude', 'longitude']].drop_duplicates()
address_duplicated = temp.groupby(
    by=['latitude', 'longitude'], dropna=False)\
    .filter(lambda x: len(x) > 1).sort_values(by=['latitude', 'address'], ascending=False)

address_duplicated
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#    display(address_duplicated)

,address,latitude,longitude
25889,1632 W HOWARD ST,42.019490,-87.672057
83917,1632 W HOWARD BLDG,42.019490,-87.672057
148429,7410 N CLARK ST,42.016328,-87.675398
77277,7410 N CLARK BLDG,42.016328,-87.675398
5653,7318 N OAKLEY AVE,42.014660,-87.687677
...,...,...,...
23977,106 W 119TH ST,41.678105,-87.625439
5574,12201-12209 S HALSTED ST,41.672270,-87.641576
37884,12201 S HALSTED ST,41.672270,-87.641576
100604,13703 S INDIANA BLDG,41.646554,-87.617083


**[cleaning]**  
Let's clean up the "address" column for these situations. We'll just use the address from the first record for the duplicated records:

In [25]:
address_dedup = address_duplicated.drop_duplicates(subset=['latitude', 'longitude'], keep='first')
address_dedup

,address,latitude,longitude
25889,1632 W HOWARD ST,42.019490,-87.672057
148429,7410 N CLARK ST,42.016328,-87.675398
5653,7318 N OAKLEY AVE,42.014660,-87.687677
11421,7138-7140 N CLARK ST,42.012561,-87.674774
98091,3146 W TOUHY BLDG,42.011980,-87.708534
...,...,...,...
15741,555-557 W 115TH ST,41.685036,-87.637083
20901,11919 S AVENUE O AVE FRNT,41.678517,-87.539894
41055,106-108 W 119TH ST,41.678105,-87.625439
5574,12201-12209 S HALSTED ST,41.672270,-87.641576


In [26]:
address_to_check, la_to_check, lo_to_check = list(set(address_duplicated['address'])), list(set(address_duplicated['latitude'])), list(set(address_duplicated['longitude']))

def clean_address(x):
    address, la, lo = x['address'], x['latitude'], x['longitude']
    if (address in address_to_check) and (la in la_to_check) and (lo in lo_to_check):
        temp = address_dedup[(address_dedup['latitude']==la) & (address_dedup['longitude']==lo)]
        if len(temp) == 1:
            address = temp['address'].values[0]

    return address
        
input_data['address'] = input_data.apply(clean_address, axis=1)
input_data

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JETS PIZZA,JETS PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153805,67732,WOLCOTTS,TROQUET,1992039,restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.961606,-87.675967
153806,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,restaurant,Risk 2 (Medium),100 W RANDOLPH ST STE C-19,CHICAGO,IL,60601,2010-01-04,tag removal,Pass,NaN,41.884586,-87.631010
153807,52234,CAFE 608,CAFE 608,2013328,restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,license re-inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.938007,-87.644755
153808,70269,MRDANIELS,MRDANIELS,1899292,restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.938443,-87.768318


Confirm the change:

In [27]:
temp = input_data[['address', 'latitude', 'longitude']].drop_duplicates()
temp.groupby(
    by=['latitude', 'longitude'], dropna=False)\
    .filter(lambda x: len(x) > 1).sort_values(by=['latitude', 'address'], ascending=False)

,address,latitude,longitude


**[exploaration]**  
Further check the "city" information:

In [28]:
temp = input_data.groupby(by=['latitude', 'longitude', 'city'], dropna=False).size().rename('count').reset_index()
duplicated_city = temp.groupby(by=['latitude', 'longitude'], dropna=False)\
    .filter(lambda x: len(x) > 1).sort_values(by=['latitude', 'longitude', 'count'], ascending=False)
duplicated_city

,latitude,longitude,city,count
15690,42.008536,-87.914428,CHICAGO,1874
15691,42.008536,-87.914428,NaN,10
14419,41.975801,-87.713499,CHICAGO,11
14420,41.975801,-87.713499,NaN,1
14271,41.973068,-87.659926,NaN,2
14270,41.973068,-87.659926,CHICAGO,1
14215,41.971217,-87.659582,CHICAGO,4
14216,41.971217,-87.659582,NaN,1
14003,41.968491,-87.659816,CHICAGO,6
14004,41.968491,-87.659816,NaN,2


**[cleaning]**  
Let's clean up the "city" column:  
- Infer and add back missing values: For locations that have a certain city and "NaN", we'll replace the "NaN" with the city.  
- For conflicting values: For locations that have multiple non-"NaN" cities, we'll choose the one with a higher count.

In [29]:
city_dedup = duplicated_city[~pd.isnull(duplicated_city['city'])].drop_duplicates(subset=['latitude', 'longitude']).drop(columns='count')
city_dedup

,latitude,longitude,city
15690,42.008536,-87.914428,CHICAGO
14419,41.975801,-87.713499,CHICAGO
14270,41.973068,-87.659926,CHICAGO
14215,41.971217,-87.659582,CHICAGO
14003,41.968491,-87.659816,CHICAGO
13670,41.965865,-87.693871,CHICAGO
13593,41.965212,-87.663260,CHICAGO
13270,41.960675,-87.729113,CHICAGO
13172,41.958548,-87.786919,CHICAGO
13102,41.956117,-87.679966,CHICAGO


In [30]:
cities_to_check, la_to_check, lo_to_check = list(set(duplicated_city['city'])), list(set(duplicated_city['latitude'])), list(set(duplicated_city['longitude']))

def clean_city(x):
    city, la, lo = x['city'], x['latitude'], x['longitude']
    if (pd.isnull(city) or (city in cities_to_check)) and (la in la_to_check) and (lo in lo_to_check):
        temp = city_dedup[(city_dedup['latitude']==la) & (city_dedup['longitude']==lo)]
        if len(temp) == 1:
            city = temp['city'].values[0]

    return city
        
input_data['city'] = input_data.apply(clean_city, axis=1)
input_data

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JETS PIZZA,JETS PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153805,67732,WOLCOTTS,TROQUET,1992039,restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.961606,-87.675967
153806,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,restaurant,Risk 2 (Medium),100 W RANDOLPH ST STE C-19,CHICAGO,IL,60601,2010-01-04,tag removal,Pass,NaN,41.884586,-87.631010
153807,52234,CAFE 608,CAFE 608,2013328,restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,license re-inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.938007,-87.644755
153808,70269,MRDANIELS,MRDANIELS,1899292,restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.938443,-87.768318


Confirm the change:

In [31]:
temp = input_data.groupby(by=['latitude', 'longitude', 'city'], dropna=False).size().rename('count').reset_index()
temp.groupby(by=['latitude', 'longitude'], dropna=False)\
    .filter(lambda x: len(x) > 1).sort_values(by=['latitude', 'longitude', 'count'], ascending=False)

,latitude,longitude,city,count


**[exploaration]**  
Further check the "zip" information:

In [32]:
temp = input_data.groupby(by=['latitude', 'longitude', 'zip'], dropna=False).size().rename('count').reset_index()
duplicated_zip = temp.groupby(by=['latitude', 'longitude'], dropna=False)\
    .filter(lambda x: len(x) > 1).sort_values(by=['latitude', 'longitude', 'count'], ascending=False)
duplicated_zip

,latitude,longitude,zip,count
15719,42.008536,-87.914428,60666,1881
15718,42.008536,-87.914428,60606,3
14556,41.977421,-87.654869,60613,1
14557,41.977421,-87.654869,60640,1
14446,41.975801,-87.713499,60625,11
...,...,...,...,...
1719,41.751592,-87.568606,NaN,1
1629,41.751030,-87.613649,60619,5
1630,41.751030,-87.613649,NaN,1
7,41.648818,-87.617080,60827,2


In [33]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(duplicated_zip)

,latitude,longitude,zip,count
15719,42.008536,-87.914428,60666,1881
15718,42.008536,-87.914428,60606,3
14556,41.977421,-87.654869,60613,1
14557,41.977421,-87.654869,60640,1
14446,41.975801,-87.713499,60625,11
14447,41.975801,-87.713499,NaN,1
14243,41.971217,-87.659582,60640,4
14244,41.971217,-87.659582,NaN,1
14031,41.968491,-87.659816,60640,6
14032,41.968491,-87.659816,NaN,2


**[cleaning]**  
Let's clean up the "zip" column:  
- Infer and add back missing values: For locations that have a certain zip and "NaN", we'll replace the "NaN" with the zip.  
- For conflicting values: For locations that have multiple non-"NaN" zip values, we'll choose the one with a higher count.

In [34]:
zip_dedup = duplicated_zip[~pd.isnull(duplicated_zip['zip'])].drop_duplicates(subset=['latitude', 'longitude']).drop(columns='count')
zip_dedup

,latitude,longitude,zip
15719,42.008536,-87.914428,60666
14556,41.977421,-87.654869,60613
14446,41.975801,-87.713499,60625
14243,41.971217,-87.659582,60640
14031,41.968491,-87.659816,60640
13698,41.965865,-87.693871,60625
13621,41.965212,-87.663260,60640
13201,41.958548,-87.786919,60634
13132,41.956117,-87.679966,60618
12560,41.948005,-87.664186,60613


In [35]:
zip_to_check, la_to_check, lo_to_check = list(set(duplicated_zip['zip'])), list(set(duplicated_zip['latitude'])), list(set(duplicated_zip['longitude']))

def clean_zip(x):
    zip_, la, lo = x['zip'], x['latitude'], x['longitude']
    if (pd.isnull(zip_) or (zip_ in zip_to_check)) and (la in la_to_check) and (lo in lo_to_check):
        temp = zip_dedup[(zip_dedup['latitude']==la) & (zip_dedup['longitude']==lo)]
        if len(temp) == 1:
            zip_ = temp['zip'].values[0]

    return zip_
        
input_data['zip'] = input_data.apply(clean_zip, axis=1)
input_data

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JETS PIZZA,JETS PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153805,67732,WOLCOTTS,TROQUET,1992039,restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.961606,-87.675967
153806,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,restaurant,Risk 2 (Medium),100 W RANDOLPH ST STE C-19,CHICAGO,IL,60601,2010-01-04,tag removal,Pass,NaN,41.884586,-87.631010
153807,52234,CAFE 608,CAFE 608,2013328,restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,license re-inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.938007,-87.644755
153808,70269,MRDANIELS,MRDANIELS,1899292,restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.938443,-87.768318


Confirm the change:

In [36]:
temp = input_data.groupby(by=['latitude', 'longitude', 'zip'], dropna=False).size().rename('count').reset_index()
temp.groupby(by=['latitude', 'longitude'], dropna=False)\
    .filter(lambda x: len(x) > 1).sort_values(by=['latitude', 'longitude', 'count'], ascending=False)

,latitude,longitude,zip,count


**[exploaration]**  
Check whether a certain zip code always have the same set of city and state:  

In [37]:
temp = input_data[~input_data['zip'].isnull()].groupby(by=['zip', 'state', 'city'], dropna=False).size().rename('count').reset_index()
duplicated_city_state = temp.groupby(by='zip', dropna=False)\
    .filter(lambda x: len(x) > 1).sort_values(by=['zip', 'count'], ascending=False)
duplicated_city_state

,zip,state,city,count
98,60655,IL,CHICAGO,652
97,60655,IL,ALSIP,1
91,60649,IL,CHICAGO,1920
90,60649,IL,CHARLES A HAYES,6
92,60649,IL,NaN,2
87,60646,IL,CHICAGO,1143
88,60646,IL,NaN,4
84,60644,IL,CHICAGO,1967
85,60644,IL,NaN,2
80,60642,IL,CHICAGO,1712


**[cleaning]**  
Let's further clean up the "state" and "city" columns according to the "zip" column:  
- All of these zip codes have non-"NaN" state or city values, we'll just trust the one with the highest count.

In [38]:
city_state_dedup = duplicated_city_state.drop_duplicates(subset=['zip']).drop(columns='count')
city_state_dedup

,zip,state,city
98,60655,IL,CHICAGO
91,60649,IL,CHICAGO
87,60646,IL,CHICAGO
84,60644,IL,CHICAGO
80,60642,IL,CHICAGO
76,60639,IL,CHICAGO
71,60633,IL,CHICAGO
58,60622,IL,CHICAGO
53,60618,IL,CHICAGO
48,60614,IL,CHICAGO


In [39]:
zip_to_check_2 = list(city_state_dedup['zip'])

def clean_city_state(x):
    zip_, city, state = x['zip'], x['city'], x['state']
    if zip_ in zip_to_check_2:
        temp = city_state_dedup[city_state_dedup['zip']==zip_]
        if len(temp) == 1:
            city = temp['city'].values[0]
            state = temp['state'].values[0]

    return city, state
        
input_data[['city', 'state']] = input_data.apply(clean_city_state, axis=1, result_type="expand")
input_data

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JETS PIZZA,JETS PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153805,67732,WOLCOTTS,TROQUET,1992039,restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.961606,-87.675967
153806,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,restaurant,Risk 2 (Medium),100 W RANDOLPH ST STE C-19,CHICAGO,IL,60601,2010-01-04,tag removal,Pass,NaN,41.884586,-87.631010
153807,52234,CAFE 608,CAFE 608,2013328,restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,license re-inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.938007,-87.644755
153808,70269,MRDANIELS,MRDANIELS,1899292,restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.938443,-87.768318


Confirm the change:

In [40]:
temp = input_data[~input_data['zip'].isnull()].groupby(by=['zip', 'state', 'city'], dropna=False).size().rename('count').reset_index()
temp.groupby(by='zip', dropna=False)\
    .filter(lambda x: len(x) > 1).sort_values(by=['zip', 'count'], ascending=False)

,zip,state,city,count


## 1.4 Clean up conflicting risk information

**[exploration]**  
Check the risk column:  
For the same facility, do they have conflicting risk status?  
Note that it is possible the risk level could change by time. I checked and confirmed that even on the same date, most of the following records still have conflicting risk status.  

In [41]:
temp = input_data[['license_id', 'name_dba', 'name_aka', 'address', 'risk']].drop_duplicates()
duplicated_risk = temp[temp.duplicated(subset=['license_id', 'name_dba', 'name_aka', 'address'], keep=False)].sort_values(by='license_id')
duplicated_risk

,license_id,name_dba,name_aka,address,risk
127712,0,FLAV-O-RICIOUS,FLAV-O-RICIOUS,600 E GRAND AVE,Risk 3 (Low)
127627,0,FLAV-O-RICIOUS,FLAV-O-RICIOUS,600 E GRAND AVE,Risk 2 (Medium)
35736,1578736,MT ARARAT DAYCARE CENTER,MT ARARAT DAYCARE CENTER,6514-6516 W HIGGINS AVE,Risk 1 (High)
35751,1578736,MT ARARAT DAYCARE CENTER,MT ARARAT DAYCARE CENTER,6514-6516 W HIGGINS AVE,Risk 3 (Low)
85087,1579115,TURKISH CUISINE AND BAKERY,TURKISH CUISINE AND BAKERY,5605-5609 N CLARK ST,Risk 3 (Low)
33799,1579115,TURKISH CUISINE AND BAKERY,TURKISH CUISINE AND BAKERY,5605-5609 N CLARK ST,Risk 1 (High)
131692,1739775,ROTISSERIE ENT & ANNETS,ROTISSERIE ENT & ANNETS,2009 N BISSELL ST,Risk 2 (Medium)
131690,1739775,ROTISSERIE ENT & ANNETS,ROTISSERIE ENT & ANNETS,2009 N BISSELL ST,Risk 3 (Low)
115550,1947909,SUBWAY,SUBWAY,6450 S PULASKI RD,Risk 3 (Low)
2313,1947909,SUBWAY,SUBWAY,6450 S PULASKI RD,Risk 1 (High)


**[cleaning]**  
Let's clean up the "risk" column for these situations:  
- Infer and add back missing values: For facilities that have a certain level and "NaN", we'll replace the "NaN" with the risk level.  
- Make conflicting values invalid: For facilities that have multiple non-"NaN" levels, we'll replace all of them with "NaN".

In [42]:
def combine_risk(x):
    non_nan_risks = list(set(x) - set([np.nan]))
    return non_nan_risks[0] if len(non_nan_risks) == 1 else np.nan

risk_dedup = duplicated_risk.groupby(by=['license_id', 'name_dba', 'name_aka', 'address'], dropna=False)['risk'].agg(combine_risk).rename('risk').reset_index()
risk_dedup

,license_id,name_dba,name_aka,address,risk
0,0,FLAV-O-RICIOUS,FLAV-O-RICIOUS,600 E GRAND AVE,NaN
1,1578736,MT ARARAT DAYCARE CENTER,MT ARARAT DAYCARE CENTER,6514-6516 W HIGGINS AVE,NaN
2,1579115,TURKISH CUISINE AND BAKERY,TURKISH CUISINE AND BAKERY,5605-5609 N CLARK ST,NaN
3,1739775,ROTISSERIE ENT & ANNETS,ROTISSERIE ENT & ANNETS,2009 N BISSELL ST,NaN
4,1947909,SUBWAY,SUBWAY,6450 S PULASKI RD,NaN
5,1954774,WELLS STREET POPCORN,WELLS STREET POPCORN,2804 N CLARK ST,NaN
6,1974918,ATINOS PIZZA,ATINOS PIZZA,4610A W DIVERSEY AVE,NaN
7,2013962,KENDALL COLLEGE,LAB KITCHEN B-ROOM 120B,900 N NORTH BRANCH ST,NaN
8,2031677,PORK SHOPPE,PORK SHOPPE,2755 W BELMONT AVE,NaN
9,2032230,MORE CUPCAKES,MORE CUPCAKES,1 E DELAWARE PL,NaN


In [43]:
license_to_check, dba_to_check, address_to_check = list(risk_dedup['license_id']), list(risk_dedup['name_dba']), list(risk_dedup['address'])

def clean_risk(x):
    license, dba, aka, address, risk = x['license_id'], x['name_dba'], x['name_aka'], x['address'], x['risk']
    if (license in license_to_check) and (dba in dba_to_check) and (address in address_to_check):
        temp = risk_dedup[(risk_dedup['license_id']==license) &
                     (risk_dedup['name_dba']==dba) &
                     (pd.isnull(risk_dedup['name_aka']) | (risk_dedup['name_aka']==aka)) &
                     (risk_dedup['address']==address)]
        if len(temp) == 1:
            risk = temp['risk'].values[0]

    return risk
        
input_data['risk'] = input_data.apply(clean_risk, axis=1)
input_data

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JETS PIZZA,JETS PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153805,67732,WOLCOTTS,TROQUET,1992039,restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.961606,-87.675967
153806,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,restaurant,Risk 2 (Medium),100 W RANDOLPH ST STE C-19,CHICAGO,IL,60601,2010-01-04,tag removal,Pass,NaN,41.884586,-87.631010
153807,52234,CAFE 608,CAFE 608,2013328,restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,license re-inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.938007,-87.644755
153808,70269,MRDANIELS,MRDANIELS,1899292,restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.938443,-87.768318


Confirm the change:

In [44]:
temp = input_data[['license_id', 'name_dba', 'name_aka', 'address', 'risk']].drop_duplicates()
temp[temp.duplicated(subset=['license_id', 'name_dba', 'name_aka', 'address'], keep=False)].sort_values(by='license_id')

,license_id,name_dba,name_aka,address,risk


## 1.5 Initial cleanup of results and violations

**[exploration]**  
Check the results and violations columns:

In [45]:
input_data['results'].value_counts()

Pass                    90506
Fail                    29845
Pass w/ Conditions      14530
Out of Business         13794
No Entry                 4257
Not Ready                 818
Business Not Located       60
Name: results, dtype: int64

In [46]:
conclusive_results = ['Pass', 'Fail', 'Pass w/ Conditions']
non_conclusive_results = ['Out of Business', 'No Entry', 'Not Ready', 'Business Not Located']

If the result is non-conclusive, then there shouldn't be violations information. But there are still such records:

In [47]:
input_data[input_data['results'].isin(non_conclusive_results) & (~input_data['violations'].isnull())]

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.886577,-87.665328
11,2079076,THE ART OF CHICKEN,THE ART OF CHICKEN,2202818,restaurant,Risk 1 (High),2041 N WESTERN AVE,CHICAGO,IL,60647,2017-08-25,complaint,No Entry,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.918742,-87.687301
38,2079101,THE PARTY ROOM,THE PARTY ROOM,2534843,catering,Risk 1 (High),10232 S VINCENNES AVE,CHICAGO,IL,60643,2017-08-25,license re-inspection,Not Ready,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.707675,-87.655848
129,2078837,SILOM12,SILOM12 THAI BUCKTOWN,2064938,restaurant,Risk 1 (High),1846-1848 N MILWAUKEE AVE,CHICAGO,IL,60647,2017-08-22,canvass,No Entry,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.914901,-87.684667
316,2078880,PARTS AND LABOR,PARTS AND LABOR,2240274,restaurant,Risk 1 (High),2700 N MILWAUKEE AVE,CHICAGO,IL,60647,2017-08-15,canvass,No Entry,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.930191,-87.709475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109860,1215137,BUBS GYROS,BUBS GYROS,2069295,restaurant,Risk 1 (High),5800 W IRVING PARK RD,CHICAGO,IL,60634,2012-05-24,complaint re-inspection,Out of Business,24. DISH WASHING FACILITIES: PROPERLY DESIGNED...,41.953228,-87.771956
115737,545787,HONEY BEAR NURSERY & KINDERGAR,HONEY BEAR NURSERY & KINDERGAR,5664,daycare,Risk 1 (High),419 W 95TH ST,CHICAGO,IL,60628,2012-01-31,canvass,Out of Business,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.721474,-87.634264
117134,545725,THE BURGER BAR,THE BURGER BAR,2064712,restaurant,Risk 2 (Medium),622 E 71ST ST,CHICAGO,IL,60619,2012-01-03,complaint re-inspection,Out of Business,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.765919,-87.609521
127908,517377,KITH & KIN,WEBSTER STREET CAFE,1799703,restaurant,Risk 1 (High),1119-1121 W WEBSTER AVE,CHICAGO,IL,60614,2011-06-01,complaint re-inspection,Out of Business,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921577,-87.656727


**[cleaning]**  
Let's clean up these violations:  

In [48]:
input_data['violations'] = input_data.apply(lambda x: np.nan if (x['results'] in non_conclusive_results) else x['violations'], axis=1)

Confirm the change:

In [49]:
input_data[input_data['results'].isin(non_conclusive_results) & (~input_data['violations'].isnull())]

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude


## 1.6 Inspection IDs and duplicated inspections

**[exploration]**  
Check that all inspection_ids are unique:

In [50]:
len(set(input_data['inspection_id'])) == len(list(input_data['inspection_id']))

True

**[exploration]**  
However, if we ignore the inspection_id, there seem to be duplicated records:

In [51]:
input_data[input_data[list(input_data)[1:]].duplicated(keep=False)]

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
2084,2069297,PICOSITO GRILL-BAR,PICOSITO GRILL-BAR,2158267,restaurant,Risk 1 (High),3553-3559 E 100TH ST,CHICAGO,IL,60617,2017-06-21,canvass,No Entry,NaN,41.713767,-87.536796
2138,2064677,PICOSITO GRILL-BAR,PICOSITO GRILL-BAR,2158267,restaurant,Risk 1 (High),3553-3559 E 100TH ST,CHICAGO,IL,60617,2017-06-21,canvass,No Entry,NaN,41.713767,-87.536796
3648,2015324,METAL HAVEN GRILL,METAL HAVEN GRILL,2457868,restaurant,Risk 1 (High),3835 W FULLERTON AVE,CHICAGO,IL,60647,2017-05-22,canvass,Out of Business,NaN,41.924416,-87.723320
3694,2050813,METAL HAVEN GRILL,METAL HAVEN GRILL,2457868,restaurant,Risk 1 (High),3835 W FULLERTON AVE,CHICAGO,IL,60647,2017-05-22,canvass,Out of Business,NaN,41.924416,-87.723320
7507,2010211,LOOMIS FOOD MARKET,LOOMIS FOOD MARKET,2004423,grocery,Risk 2 (Medium),6859 S LOOMIS BLVD,CHICAGO,IL,60636,2017-03-14,complaint,No Entry,NaN,41.768716,-87.658922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152903,68271,DE ARCOS PIZZA & RESTAURANT,DE ARCOS PIZZA & RESTAURANT,1577283,restaurant,Risk 1 (High),2832 E 87TH ST,CHICAGO,IL,60617,2010-01-22,tag removal,Pass,NaN,41.737447,-87.555039
153108,88223,ASPIRA HAUGAN,ASPIRA HAUGAN MIDDLE SCHOOL,66351,school,Risk 1 (High),3729 W LELAND AVE,CHICAGO,IL,60625,2010-01-20,canvass,Pass,NaN,41.966378,-87.721826
153109,88222,ASPIRA HAUGAN,ASPIRA HAUGAN MIDDLE SCHOOL,66351,school,Risk 1 (High),3729 W LELAND AVE,CHICAGO,IL,60625,2010-01-20,canvass,Pass,NaN,41.966378,-87.721826
153113,158274,ARGENTINA FOODS,ARGENTINA FOODS,57047,grocery,Risk 2 (Medium),4500 S WOOD ST,CHICAGO,IL,60609,2010-01-20,out of business,Fail,NaN,41.812105,-87.670072


**[cleaning]**  
There are 175 records that contain duplicated information. For example, inspection #2069297 and #2064677 are exactly the same except for the inspection_id itself. For the purpose of our target use case, we don't need those repeated records, so we'll remove them:

In [52]:
input_data = input_data.drop_duplicates(subset=list(input_data)[1:], ignore_index=True)
input_data

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,NaN,41.853651,-87.620534
1,2079129,JETS PIZZA,JETS PIZZA,2522268,restaurant,Risk 2 (Medium),1025 W MADISON ST,CHICAGO,IL,60607,2017-08-28,license,Not Ready,NaN,41.881572,-87.653052
2,2079125,ROOM 1520,ROOM 1520,2446638,special event,Risk 3 (Low),301 N JUSTINE ST,CHICAGO,IL,60607,2017-08-28,license re-inspection,Not Ready,NaN,41.886577,-87.665328
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338,restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616,2017-08-28,license,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079,school,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639,2017-08-28,license re-inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153712,67732,WOLCOTTS,TROQUET,1992039,restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.961606,-87.675967
153713,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,restaurant,Risk 2 (Medium),100 W RANDOLPH ST STE C-19,CHICAGO,IL,60601,2010-01-04,tag removal,Pass,NaN,41.884586,-87.631010
153714,52234,CAFE 608,CAFE 608,2013328,restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,license re-inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.938007,-87.644755
153715,70269,MRDANIELS,MRDANIELS,1899292,restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,license re-inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.938443,-87.768318


**[exploration]** 

In [53]:
num_inspection = len(input_data)
print(f"How many inspection records are there in total? {num_inspection}")
num_license = len(set(input_data['license_id']))
print(f"How many unique license numbers are there? {num_license}")
num_facility = len(input_data[['license_id', 'name_dba', 'name_aka', 'facility_type', 'address']].drop_duplicates())
print(f"How many unique food facilities are there? {num_facility}")
num_dates = len(set(input_data['inspection_date']))
print(f"How many unique inspection dates are there? {num_dates}")

How many inspection records are there in total? 153717
How many unique license numbers are there? 32851
How many unique food facilities are there? 34037
How many unique inspection dates are there? 1946


# 2. Try to better understand the data

## 2.1 What defines a facility?

**[exploration]**  
### 1. The same license_id can have multiple name_dba:

In [54]:
temp = input_data[(input_data['license_id']!='') & (input_data['license_id']!='nan') & (~pd.isnull(input_data['license_id']))][[
    'license_id', 'name_dba']].drop_duplicates()
temp[temp.duplicated(subset='license_id', keep=False)].sort_values(by='license_id')

,license_id,name_dba
108764,0,NORMAS MINI STORE
107063,0,KENNY
135814,0,GRILL IN THE PARK
107553,0,BABYCAKES GOURMET LLC
135473,0,EDGEWATER FOOD PANTRY
...,...,...
17349,87119,PUB ROYALE
19210,9648,CAFFE ITALIA
15801,9648,CAFE #10
134717,968,CAFE BERNARD


### 2. The same license_id can have multiple addresses:

In [55]:
temp = input_data[(input_data['license_id']!='') & (input_data['license_id']!='nan') & (~pd.isnull(input_data['license_id']))][[
    'license_id', 'address', 'city', 'state', 'zip', 'latitude','longitude']].drop_duplicates()
temp[temp.duplicated(subset='license_id', keep=False)].sort_values(by='license_id')

,license_id,address,city,state,zip,latitude,longitude
216,0,4225 N CENTRAL AVE,CHICAGO,IL,60634,41.957639,-87.767001
105890,0,512 E 71ST ST,CHICAGO,IL,60619,41.765871,-87.612286
18068,0,4807 N DAMEN AVE,CHICAGO,IL,60625,41.969019,-87.679045
105565,0,300 E 47TH,CHICAGO,IL,60653,41.809447,-87.619524
105170,0,2157 W 19TH ST,CHICAGO,IL,60608,41.855674,-87.680646
...,...,...,...,...,...,...,...
3102,70217,2550 W 56TH (5600S) ST,CHICAGO,IL,60629,41.792067,-87.688478
9875,80130,179 W VAN BUREN ST,CHICAGO,IL,60605,41.876748,-87.633286
68628,80130,111 E WACKER DR FL,CHICAGO,IL,60601,41.888079,-87.624047
7126,81030,1461 E HYDE PARK BLVD,CHICAGO,IL,60615,41.802341,-87.589372


Note: even the same set of 'license_id', 'name_dba', 'name_aka', 'facility_type' can still have different address/latitude/longitude:

In [56]:
temp = input_data[(input_data['license_id']!='') & (input_data['license_id']!='nan') & (~pd.isnull(input_data['license_id']))][[
    'license_id', 'name_dba', 'name_aka', 'facility_type', 'address', 'city', 'state', 'zip', 'latitude','longitude']].drop_duplicates()
temp[temp.duplicated(subset=['license_id', 'name_dba', 'name_aka', 'facility_type'], keep=False)].sort_values(by='license_id')

,license_id,name_dba,name_aka,facility_type,address,city,state,zip,latitude,longitude
117243,0,UNKNOWN,NaN,NaN,8090 S COMMERCIAL AVE,CHICAGO,IL,60617,NaN,NaN
136021,0,DUNKIN DONUTS,NaN,NaN,2640 W DIVERSEY AVE,CHICAGO,IL,60647,41.932226,-87.694274
105565,0,UNKNOWN,NaN,NaN,300 E 47TH,CHICAGO,IL,60653,41.809447,-87.619524
56841,0,HERBAL LIFE,HERBAL LIFE,herbal,1854 W 18TH ST BLDG,CHICAGO,IL,60608,41.857806,-87.673279
18068,0,HERBAL LIFE,HERBAL LIFE,herbal,4807 N DAMEN AVE,CHICAGO,IL,60625,41.969019,-87.679045
122919,0,UNKNOWN,NaN,NaN,10222 S WENTWORTH AVE,CHICAGO,IL,60628,41.708213,-87.628492
124607,0,DUNKIN DONUTS,NaN,NaN,8700 S EXCHANGE AVE,CHICAGO,IL,60617,41.737270,-87.552955
3425,1142125,POTBELLY SANDWICH WORKS LLC,POTBELLY SANDWICH WORKS LLC,restaurant,5700 S CICERO AVE,CHICAGO,IL,60638,41.789329,-87.741646
77705,1142125,POTBELLY SANDWICH WORKS LLC,POTBELLY SANDWICH WORKS LLC,restaurant,5800 S CICERO AVE,CHICAGO,IL,60638,41.787652,-87.741560
3196,13976,ALDI INC #64,ALDI INC #64,grocery,1753 N MILWAUKEE AVE,CHICAGO,IL,60647,41.913342,-87.681822


### 3. The same license number and name_dba can have multiple name_akas.   
For example, there are 81 distinct name_dba/facility type combinations for license_id14616 ILLINOIS SPORTSERVICE INC:

In [57]:
temp = input_data[['license_id', 'name_dba', 'name_aka', 'facility_type']].drop_duplicates().groupby(
    by=['license_id', 'name_dba'], dropna=False).size().rename('count').reset_index()\
    .sort_values(by='count', ascending=False).reset_index(drop=True)
temp.head(5)

,license_id,name_dba,count
0,14616,ILLINOIS SPORTSERVICE INC,81
1,1974745,THE UNITED CENTER,45
2,1354323,SPORTSERVICE SOLDIER FIELD,45
3,1574001,LEVY RESTAURANTS AT WRIGLEY FIELD,19
4,1932251,MCCORMICK PLACE,16


There are 178 such license_id/name_dba combinations that have more than 1 name_dba/facility type combinations:

In [58]:
len(temp[temp['count']>1])

178

Take a closer look at license_id14616:

In [59]:
input_data[(input_data['license_id']=='14616') & (input_data['name_dba']=='ILLINOIS SPORTSERVICE INC')][
    ['license_id', 'name_dba', 'name_aka', 'facility_type']].drop_duplicates()

,license_id,name_dba,name_aka,facility_type
2209,14616,ILLINOIS SPORTSERVICE INC,GUARANTEED RATE FIELD,stadium
123192,14616,ILLINOIS SPORTSERVICE INC,MAIN COMMISSARY,commissary
123200,14616,ILLINOIS SPORTSERVICE INC,SHERM LOLLARS GUARD THE PLATE GRILLE STAND #524,restaurant
128176,14616,ILLINOIS SPORTSERVICE INC,TEX-MEX 3 - STEAK (#544),kiosk
128419,14616,ILLINOIS SPORTSERVICE INC,BBQ BRISKET (#531),kiosk
...,...,...,...,...
149924,14616,ILLINOIS SPORTSERVICE INC,DIPPING DOTS (SEC #132),restaurant
149950,14616,ILLINOIS SPORTSERVICE INC,GO GO WHITE SOX,restaurant
149955,14616,ILLINOIS SPORTSERVICE INC,NACHO 8 (#544),restaurant
149973,14616,ILLINOIS SPORTSERVICE INC,HOT DOG VIENNA BEEF 7 (#557),restaurant


### 4. The same address can have multiple license numbers

In [60]:
temp = input_data[(input_data['license_id']!='') & (input_data['license_id']!='nan') & (~pd.isnull(input_data['license_id']))][[
    'license_id', 'address', 'city', 'state', 'zip', 'latitude','longitude']].drop_duplicates()
temp[temp.duplicated(subset=['address', 'city', 'state', 'zip', 'latitude','longitude'], keep=False)].sort_values(by='address')

,license_id,address,city,state,zip,latitude,longitude
28429,2442868,,NaN,IL,NaN,NaN,NaN
149587,0,,NaN,IL,NaN,NaN,NaN
10019,2215538,1 E 113TH ST,CHICAGO,IL,60628,41.688848,-87.622894
116232,21702,1 E 113TH ST,CHICAGO,IL,60628,41.688848,-87.622894
116234,1800876,1 E 113TH ST,CHICAGO,IL,60628,41.688848,-87.622894
...,...,...,...,...,...,...,...
32207,2113336,9917 S EWING AVE,CHICAGO,IL,60617,41.715244,-87.535130
75412,2046663,9928 S CRANDON AVE FL,CHICAGO,IL,60617,41.714402,-87.567167
9089,3247017,9928 S CRANDON AVE FL,CHICAGO,IL,60617,41.714402,-87.567167
83366,2093742,9939 S HALSTED ST,CHICAGO,IL,60628,41.713035,-87.642832


### 5. Even the same (?) food facility can have different license numbers!

In [61]:
temp = input_data[[
    'name_dba','name_aka','license_id','facility_type','address','city','state','zip',
    'latitude','longitude']].drop_duplicates()
facility_counts = temp.groupby(
    by=['name_dba','name_aka','facility_type','address','city','state','zip','latitude','longitude'], dropna=False)\
    .size().rename('count').reset_index().sort_values(by='count', ascending=False)
facility_counts.head(10)

,name_dba,name_aka,facility_type,address,city,state,zip,latitude,longitude,count
27180,TRIPLE A SERVICES INC,TRIPLE A SERVICES INC,mobile food preparer,2637 S THROOP ST FL,CHICAGO,IL,60608,41.844671,-87.654962,43
19437,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,16
27671,VHC LOWER LLC,3RD FLOOR - BANQUET/MEETING ROOMS(LL KITCHEN),restaurant,203 N WABASH AVE,CHICAGO,IL,60601,41.885828,-87.626138,15
19379,PALETERIA AZTECA #2,NaN,mobile food preparer,3119 W CERMAK RD,CHICAGO,IL,60623,41.851676,-87.703696,10
23287,SOHO HOUSE,SOHO HOUSE,restaurant,113-125 N GREEN ST,CHICAGO,IL,60607,41.883557,-87.648432,10
27894,WALDORF ASTORIA CHICAGO,WALDORF ASTORIA CHICAGO,restaurant,11 E WALTON ST,CHICAGO,IL,60611,41.899835,-87.627799,9
9515,FLAT TOP GRILL,FLAT TOP GRILL,restaurant,30 S WABASH AVE,CHICAGO,IL,60603,41.881335,-87.626335,8
19376,PALETERIA AZTECA #2,PALETERIA AZTECA #2,mobile food preparer,3119 W CERMAK RD,CHICAGO,IL,60623,41.851676,-87.703696,7
19323,PACIFIC LANGHAM CHICAGO CORPORATION,THE LANGHAM CHICAGO,restaurant,330 N WABASH AVE,CHICAGO,IL,60611,41.887939,-87.627036,7
19744,PARS ICECREAM CO INC,NaN,mobile food preparer,4825 W ARTHINGTON ST,CHICAGO,IL,60644,41.869236,-87.745893,7


In [62]:
print(f"{len(facility_counts[facility_counts['count']>1])} such facilities are involved!")
print(f"That's {round(100*len(facility_counts[facility_counts['count']>1])/len(facility_counts), 2)}% of all facilities!!")

4017 such facilities are involved!
That's 13.84% of all facilities!!


Note that some of them have the facility type of "Mobile Prepared Food Vendor". Some of the "Restaurant" facilities seem to be hotels, or within a complex.   
In the following example, most of them have the facility type of "Mobile Prepared Food Vendor":

In [63]:
input_data[input_data['name_aka']=='LA MEXICANA TAMALE CART'][
    ['license_id', 'name_dba','name_aka','facility_type','address','city','state','zip',
     'latitude','longitude','inspection_date']
].drop_duplicates()

,license_id,name_dba,name_aka,facility_type,address,city,state,zip,latitude,longitude,inspection_date
5099,2506389,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11717,2506388,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11720,2506235,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11721,2506234,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11725,2506233,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11727,2506231,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11729,2506230,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11731,2506222,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11735,2506220,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10
11737,2506221,PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,LA MEXICANA TAMALE CART,mobile food preparer,4171 S ARCHER AVE,CHICAGO,IL,60632,41.817618,-87.698258,2017-01-10


Since these facilities, although sharing the same address, are treated differently for the food inspection purpose, here we need to treat them as distinct facilities!

## 2.2 Duplicated inspections?

**[exploration]**  
### 6. Are there any redundant inspection entries on the same day for the same facility?

In [64]:
temp2 = input_data[['inspection_id', 'license_id', 'name_dba', 'name_aka', 'inspection_date', 'inspection_type']]
inspection_counts2 = temp2.groupby(by=['license_id', 'name_dba', 'name_aka', 'inspection_date', 'inspection_type'], dropna=False).size().rename('count').reset_index()\
    .sort_values(by='count', ascending=False)
inspection_counts2.head(10)

,license_id,name_dba,name_aka,inspection_date,inspection_type,count
12931,1354323,SPORTSERVICE SOLDIER FIELD,GRIDIRON GRILL,2010-06-07,canvass,5
12933,1354323,SPORTSERVICE SOLDIER FIELD,GRIDIRON GRILL,2011-05-23,canvass,3
12892,1354323,SPORTSERVICE SOLDIER FIELD,1ST DOWN FRANKS,2010-06-07,canvass,3
12899,1354323,SPORTSERVICE SOLDIER FIELD,2ND CITY PIZZA,2010-06-07,canvass,3
16591,14616,ILLINOIS SPORTSERVICE INC,MOOSES DOGS AND POLISH 10 (#538),2010-03-25,canvass,2
23926,1578830,BUFFET CASTLE,BUFFET CASTLE,2013-04-18,canvass,2
24426,1592201,6 DEGREES,6 DEGREES,2016-09-09,canvass,2
66801,2048785,TRUMP INTERNATIONAL HOTEL,SIXTEEN,2017-05-09,canvass,2
137502,3727,ALL AMERICAN NURSING HOME,ALL AMERICAN NURSING HOME,2010-02-25,canvass,2
139166,40155,POPEYES CHICKEN/INDIAN SWAAD,POPEYES CHICKEN/INDIAN SWAAD,2010-05-20,complaint re-inspection,2


In [65]:
print("How many unique food facilities/inspection date combinations have multiple inspections on that date?")
num_facility_date_multiple_inspections = len(inspection_counts2[inspection_counts2['count']>1])
num_facility_date_comb = len(inspection_counts2)
print(f"{num_facility_date_multiple_inspections} out of {num_facility_date_comb} ({round(100*num_facility_date_multiple_inspections/num_facility_date_comb, 2)}%)")

print("How many inspection records are involved ?")
num_inspection_involved = inspection_counts2[inspection_counts2['count']>1]['count'].sum()
print(f"{num_inspection_involved} out of {num_inspection} ({round(100*num_inspection_involved/num_inspection, 2)}%)")

print("How many unique license numbers are involved?")
num_license_involved = len(inspection_counts2[inspection_counts2['count']>1][['license_id']].drop_duplicates())
print(f"{num_license_involved} out of {num_license} ({round(100*num_license_involved/num_license, 2)}%)")

print("How many unique food facilities are involved?")
num_facility_involved = len(inspection_counts2[inspection_counts2['count']>1][['license_id', 'name_aka']].drop_duplicates())
print(f"{num_facility_involved} out of {num_facility} ({round(100*num_facility_involved/num_facility, 2)}%)")

print("How many unique inspection dates are involved?")
num_dates_involved = len(inspection_counts2[inspection_counts2['count']>1][['inspection_date']].drop_duplicates())
print(f"{num_dates_involved} out of {num_dates} ({round(100*num_dates_involved/num_dates, 2)}%)")

How many unique food facilities/inspection date combinations have multiple inspections on that date?
393 out of 153318 (0.26%)
How many inspection records are involved ?
792 out of 153717 (0.52%)
How many unique license numbers are involved?
370 out of 32851 (1.13%)
How many unique food facilities are involved?
381 out of 34037 (1.12%)
How many unique inspection dates are involved?
336 out of 1946 (17.27%)


What's happening there? Take one example to look into the details:

In [66]:
input_data[(input_data['license_id']=='1354323') & (input_data['name_aka']=='GRIDIRON GRILL') & (input_data['inspection_date']=='2010-06-07')]

,inspection_id,name_dba,name_aka,license_id,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude
146342,250633,SPORTSERVICE SOLDIER FIELD,GRIDIRON GRILL,1354323,restaurant,Risk 2 (Medium),1410 S MUSEUM CAMPUS DR,CHICAGO,IL,60605,2010-06-07,canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.862628,-87.615031
146350,197469,SPORTSERVICE SOLDIER FIELD,GRIDIRON GRILL,1354323,restaurant,Risk 2 (Medium),1410 S MUSEUM CAMPUS DR,CHICAGO,IL,60605,2010-06-07,canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.862628,-87.615031
146358,112420,SPORTSERVICE SOLDIER FIELD,GRIDIRON GRILL,1354323,restaurant,Risk 2 (Medium),1410 S MUSEUM CAMPUS DR,CHICAGO,IL,60605,2010-06-07,canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.862628,-87.615031
146363,160390,SPORTSERVICE SOLDIER FIELD,GRIDIRON GRILL,1354323,restaurant,Risk 2 (Medium),1410 S MUSEUM CAMPUS DR,CHICAGO,IL,60605,2010-06-07,canvass,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.862628,-87.615031
146406,197476,SPORTSERVICE SOLDIER FIELD,GRIDIRON GRILL,1354323,restaurant,Risk 2 (Medium),1410 S MUSEUM CAMPUS DR,CHICAGO,IL,60605,2010-06-07,canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.862628,-87.615031


OK. It looks like there are indeed multiple inspections on the same date. Although the results are the same ("Pass"), the detailed "violations" notes are different! In this case, **we will need to collapse these inspections**. (We'll do this in a later section!)  
Also note that some of the violations have different comments although they share the same violation code:

In [67]:
input_data.loc[146342, 'violations']

'34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: The floors shall be constructed per code, be smooth and easily cleaned, and be kept clean and in good repair. \n \nMUST DETAIL CLEAN FLOORS UNDER GRILL AND REMOVE FOOD DEBRIS AND GREASE BUILD UP.  '

In [68]:
input_data.loc[146406, 'violations']

'34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: The floors shall be constructed per code, be smooth and easily cleaned, and be kept clean and in good repair. \nSTANDING WATER NEXT TO GREASE TRAP. INSTRUCTED TO CLEAN AND MAINTAIN. | 35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: The walls and ceilings shall be in good repair and easily cleaned. \nPEELING PAINT NOTED ON WALL NEXT TO DEEP FRYER, INSTRUCTED MANAGER TO REMOVE AND MAINTAIN.'

Let's also consider address, results and violatoins:  
Now there are no duplicates!

In [69]:
temp = input_data[['inspection_id', 'license_id', 'name_dba', 'name_aka', 'facility_type', 'address', 'inspection_date', 'inspection_type', 'results', 'violations']]
temp.duplicated(subset=list(temp)[1:], keep=False).sum()

0

# 3. Further data cleanup and split into multiple tables

## 3.1 Facility and Address tables

Based on above exploration, we define a "facility" as a unique combination of 'license_id', 'name_dba', 'name_aka', 'facility_type', 'address', 'zip'. To be convenient, we need to assign an artifical id to a certain facility.

[check] Is it true that a unique "address" and "zip" combination has unique set of "city", "state", "latitude" and "longitude"? In other words, we only need the "address" and "zip" columns, and the "city", "state", "latitude" and "longitude" columns are not providing additional information in identifying a certain facility. This is true:

In [70]:
addresses = input_data[['address', 'city', 'state', 'zip', 'latitude', 'longitude']].drop_duplicates()
addresses.duplicated(subset=['address', 'zip'], keep=False).sum()

0

In [71]:
facilities = input_data[['name_dba', 'name_aka', 'license_id', 'facility_type', 'address', 'zip']].drop_duplicates().sort_values(by='name_dba').reset_index(drop=True)
facilities.insert (0, 'facility_id', facilities.index)
facilities

,facility_id,name_dba,name_aka,license_id,facility_type,address,zip
0,0,#1 CHINA EXPRESS LTD,#1 CHINA EXPRESS LTD,1869616,restaurant,4248 S WENTWORTH AVE,60609
1,1,#1 CHOP SUEY,#1 CHOP SUEY,2042372,restaurant,1907 W 87TH ST,60620
2,2,#1 CHOP SUEY,#1 CHOP SUEY,1970042,restaurant,4307 W IRVING PARK RD,60641
3,3,#1 CHOP SUEY RESTAURANT,#1 CHOP SUEY RESTAURANT,1739799,restaurant,4307 W IRVING PARK RD,60641
4,4,#1 DELI INC,31 DELI INC,2501188,grocery,6901 S NORMAL BLVD,60621
...,...,...,...,...,...,...,...
34032,34032,ZULLOS MARKETS LLC,ZULLOS MARKET,2016915,restaurant,131 N CLINTON ST,60661
34033,34033,ZUS QUICK MART,ZUS QUIICK MART,2020878,grocery,11641 S HALSTED ST,60628
34034,34034,ZUZU WRAPS INC,ZUZU WRAPS,2256140,restaurant,3200 W LAWRENCE AVE,60625
34035,34035,ZYGGYZ GRILL,ZYGGYZ GRILL,1991900,restaurant,6604 N SHERIDAN RD BLDG,60626


In [72]:
facilities.to_csv("data/tables/facilities.csv", index=False)

In [73]:
addresses = addresses[['address', 'zip', 'city', 'state', 'latitude', 'longitude']]
addresses

,address,zip,city,state,latitude,longitude
0,2121 S PRAIRIE AVE,60616,CHICAGO,IL,41.853651,-87.620534
1,1025 W MADISON ST,60607,CHICAGO,IL,41.881572,-87.653052
2,301 N JUSTINE ST,60607,CHICAGO,IL,41.886577,-87.665328
4,2245 N MCVICKER AVE,60639,CHICAGO,IL,41.921675,-87.776711
5,4355 S COTTAGE GROVE AVE,60653,CHICAGO,IL,41.815197,-87.606531
...,...,...,...,...,...,...
153489,3333 W FULLERTON AVE,60647,CHICAGO,IL,41.924554,-87.711953
153567,13205 S MUSKEGON AVE,60633,CHICAGO,IL,41.655351,-87.554280
153653,7105 S RACINE AVE,60636,CHICAGO,IL,41.764896,-87.653965
153668,2003 W 69TH ST,60636,CHICAGO,IL,41.768328,-87.673819


In [74]:
addresses.to_csv("data/tables/addresses.csv", index=False)

In [75]:
# add the facility_id column to the dataset and double check
input_data = input_data.merge(facilities)
len(set(input_data['facility_id'])) == len(facilities)

True

In [76]:
input_data.to_csv('data/half_clean/input_data_cleaned_intermediate.csv', index=False)

## 3.2 Risk and Facility_Risk tables

Here we double check that each facility has a certain risk level:

In [77]:
facility_risk = input_data[~input_data['risk'].isnull()][['facility_id', 'risk']].drop_duplicates()
facility_risk.duplicated(subset=['facility_id'], keep=False).sum()

0

In [78]:
facility_risk = facility_risk.sort_values(by='facility_id').reset_index(drop=True)
facility_risk

,facility_id,risk
0,0,Risk 1 (High)
1,1,Risk 1 (High)
2,2,Risk 1 (High)
3,3,Risk 1 (High)
4,4,Risk 2 (Medium)
...,...,...
33949,34032,Risk 2 (Medium)
33950,34033,Risk 3 (Low)
33951,34034,Risk 1 (High)
33952,34035,Risk 1 (High)


In [79]:
facility_risk['risk'].value_counts()

Risk 1 (High)      18331
Risk 3 (Low)        8403
Risk 2 (Medium)     7202
All                   18
Name: risk, dtype: int64

Let's further seperate the risk code and risk level. For "All", let's define a risk code 0 to indicate "All".

In [80]:
facility_risk['risk'] = facility_risk['risk'].str.replace('All', 'Risk 0 (All)')
facility_risk['risk'].value_counts()

Risk 1 (High)      18331
Risk 3 (Low)        8403
Risk 2 (Medium)     7202
Risk 0 (All)          18
Name: risk, dtype: int64

In [81]:
def extract_risk(s):
    matched = re.compile(r'\w+ (?P<code>\d+) \((?P<level>\w+)\)').match(s)
    if matched:
        return matched.group('code'), matched.group('level')
    else:
        return '', ''

facility_risk['risk_code'], facility_risk['risk_level'] = zip(*facility_risk['risk'].apply(extract_risk))
facility_risk

,facility_id,risk,risk_code,risk_level
0,0,Risk 1 (High),1,High
1,1,Risk 1 (High),1,High
2,2,Risk 1 (High),1,High
3,3,Risk 1 (High),1,High
4,4,Risk 2 (Medium),2,Medium
...,...,...,...,...
33949,34032,Risk 2 (Medium),2,Medium
33950,34033,Risk 3 (Low),3,Low
33951,34034,Risk 1 (High),1,High
33952,34035,Risk 1 (High),1,High


In [82]:
risks = facility_risk[['risk_code', 'risk_level']].drop_duplicates().sort_values(by='risk_code').reset_index(drop=True)\
    .rename(columns={'risk_code': 'code', 'risk_level': 'level'})
risks.to_csv("data/tables/risks.csv", index=False)
risks

,code,level
0,0,All
1,1,High
2,2,Medium
3,3,Low


In [83]:
facility_risk = facility_risk.drop(columns=['risk', 'risk_level'])
facility_risk.to_csv("data/tables/facility_risk.csv", index=False)
facility_risk

,facility_id,risk_code
0,0,1
1,1,1
2,2,1
3,3,1
4,4,2
...,...,...
33949,34032,2
33950,34033,3
33951,34034,1
33952,34035,1


## 3.3 Inspection and Facility_Inspection tables

In [84]:
inspection_results = input_data[['inspection_id', 'inspection_date', 'inspection_type', 'results', 'violations', 'facility_id']]
inspection_results

,inspection_id,inspection_date,inspection_type,results,violations,facility_id
0,2079132,2017-08-28,license,Pass,NaN,19402
1,2078553,2017-08-15,license,Not Ready,NaN,19402
2,2079129,2017-08-28,license,Not Ready,NaN,15093
3,2079125,2017-08-28,license re-inspection,Not Ready,NaN,25450
4,2072040,2017-08-08,license,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,25450
...,...,...,...,...,...,...
153712,70272,2010-01-05,out of business,Pass,NaN,30313
153713,68251,2010-01-05,license re-inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,7670
153714,158243,2010-01-05,license,Fail,NaN,9059
153715,160208,2010-01-05,out of business,Pass,NaN,18585


Check the results column:

In [85]:
inspection_results['results'].value_counts()

Pass                    90485
Fail                    29835
Pass w/ Conditions      14530
Out of Business         13737
No Entry                 4252
Not Ready                 818
Business Not Located       60
Name: results, dtype: int64

As mentioned above, there is a repeated inspection issue - for a certain facility_id/inspection_date/inspection_type combination, how many different inspections are there? We are expecting count=1 but we see count>1 in 367 such combinations and it involves 740 inspection records:

In [86]:
inspection_dup_counts = inspection_results.groupby(by=['facility_id', 'inspection_date', 'inspection_type'], dropna=False).size().rename('count').reset_index()\
    .sort_values(by='count', ascending=False)
inspection_dup_counts[inspection_dup_counts['count'] > 1]

,facility_id,inspection_date,inspection_type,count
122905,27465,2010-06-07,canvass,5
122907,27465,2011-05-23,canvass,3
122963,27484,2010-06-07,canvass,3
122984,27490,2010-06-07,canvass,3
99834,22488,2015-07-07,canvass,2
...,...,...,...,...
94601,21407,2010-10-07,canvass,2
21856,5258,2015-07-08,canvass,2
8419,2178,2010-01-06,complaint re-inspection,2
9660,2438,2013-02-07,canvass,2


Inspections involved:

In [87]:
inspections_duplicated = inspection_results[inspection_results.duplicated(subset=['facility_id', 'inspection_date', 'inspection_type'], keep=False)].sort_values(by='facility_id')
inspections_duplicated

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#    display(inspections_duplicated)

,inspection_id,inspection_date,inspection_type,results,violations,facility_id
45325,1990125,2017-03-07,canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",111
45326,1990135,2017-03-07,canvass,Out of Business,NaN,111
152367,277609,2010-07-21,canvass,Pass,"14. PREVIOUS SERIOUS VIOLATION CORRECTED, 7-42...",206
152368,277587,2010-07-21,canvass,Fail,"14. PREVIOUS SERIOUS VIOLATION CORRECTED, 7-42...",206
111358,1559937,2015-07-07,canvass,Out of Business,NaN,213
...,...,...,...,...,...,...
109645,1567176,2015-09-03,canvass,Out of Business,NaN,33866
133527,78338,2010-03-09,canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,33898
133528,78337,2010-03-09,canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,33898
113817,58461,2010-05-03,canvass,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,33989


Some examples:

In [88]:
inspection_results[(inspection_results['facility_id']==27465) & (inspection_results['inspection_date']=='2010-06-07')]

,inspection_id,inspection_date,inspection_type,results,violations,facility_id
127533,250633,2010-06-07,canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",27465
127534,197469,2010-06-07,canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,27465
127535,112420,2010-06-07,canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,27465
127536,160390,2010-06-07,canvass,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",27465
127537,197476,2010-06-07,canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",27465


In [89]:
inspection_results[(inspection_results['facility_id']==27465) & (inspection_results['inspection_date']=='2011-05-23')]

,inspection_id,inspection_date,inspection_type,results,violations,facility_id
127528,567409,2011-05-23,canvass,Pass,NaN,27465
127529,569352,2011-05-23,canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,27465
127530,565353,2011-05-23,canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",27465


In [90]:
inspection_results[(inspection_results['facility_id']==206) & (inspection_results['inspection_date']=='2010-07-21')]

,inspection_id,inspection_date,inspection_type,results,violations,facility_id
152367,277609,2010-07-21,canvass,Pass,"14. PREVIOUS SERIOUS VIOLATION CORRECTED, 7-42...",206
152368,277587,2010-07-21,canvass,Fail,"14. PREVIOUS SERIOUS VIOLATION CORRECTED, 7-42...",206


It looks like there could exist multiple inspections on the same date for the same facility. In the above examples, although some times the results are the same ("Pass"), the detailed "violations" notes could be different! Also note that some of the violations have different comments although they share the same violation code:

In [91]:
input_data.loc[146342, 'violations']

'21. * CERTIFIED FOOD MANAGER ON SITE WHEN POTENTIALLY HAZARDOUS FOODS ARE  PREPARED AND SERVED - Comments: A certified food service manager must be present in all establishments at which potentially hazardous food is prepared or served. \nNO ORIGINAL CHICAGO FOOD SANITATION CERTIFICATE POSTED. | 36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIGHT PROVIDED, FIXTURES SHIELDED - Comments: Shielding to protect against broken glass falling into food shall be provided for all artificial lighting sources in preparation, service, and display facilities. \nREPLACE DAMAGED LIGHT SHIELDS IN PREP AREA. | 38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQUIRED: PLUMBING: INSTALLED AND MAINTAINED - Comments: REPAIR LEAKING OVERHEAD PIPE IN UTILITY CLOSET.'

In [92]:
input_data.loc[146406, 'violations']

"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: No Pest Control logbook on premises by licensed pest control company. Instructed that a licensed pest control company must service premises and provided logbook with all required documents. Serious citation issued 7-38-020. | 21. * CERTIFIED FOOD MANAGER ON SITE WHEN POTENTIALLY HAZARDOUS FOODS ARE  PREPARED AND SERVED - Comments: No Certified Food Manager on premises when potentially hazardous foods are served and prepared-establishment prepares various beverages-hot and cold using dairy products. Instructed that a Certified Food Manager must be on premises during all hours of operation. Serious citation issued 7-38-012. | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: Floors along wallbase thru-out premises not cleaned. Instructed to detail clean daily. | 35. WALLS

**[solution]**   
We need to collapse these repeated inspections.   

**Rules** - For the same facility and the same inspection type, on the same date, if have more than one results:
- If have non-conflicting results (such as “Pass” and “Out of Business”), will just keep the conclusive one (such as “Pass”) and ignore the non-conclusive one (such as “Out of Business”).
- If have conflicting results (such as “Pass” and “Fail”), will consider both invalid and remove all of them.
- If have “No Entry” and other results, then remove “No Entry”.
- If have the same result (such as “Pass” or “Fail”) with different “violations”, then combine these violations.
- If have “Pass” and “Pass w/ Conditions”, then consider it as “Pass w/ Conditions”, and combine violations.
- If only have “Business not loacated” and “Out of Business”, just keep “Out of Business”.
- In all of these cases, only keep one inspection id, and add a “-c” at the end to indicate “collapsed”. 

Note:  
conclusive_results = ['Pass', 'Fail', 'Pass w/ Conditions'];  
non_conclusive_results = ['Out of Business', 'No Entry', 'Not Ready', 'Business Not Located']

In [93]:
def inspection_agg(df):
    d = {}
    debug = False
    
    def process_single_type(d, df):
        result = df['results'].values[0]
        d['results'] = result
        d['inspection_id'] = df['inspection_id'].values[0] + '-s'
        d['violations'] = np.nan
        if (result in conclusive_results) and ((~df['violations'].isnull()).sum() > 0):            
            d['violations'] = ' | '.join([violation for violation in list(set(df['violations'])) if not pd.isna(violation)])                                        
        return d

    def choose_non_conclusive_type(d, df, type_):
        df = df[df['results']==type_]
        d['results'] = type_
        d['inspection_id'] = df['inspection_id'].values[0] + '-s'
        d['violations'] = np.nan 
        return d     
    
    # remove all "No Entry" results because they are not informative
    df = df[df['results']!='No Entry']
    
    results = list(set(df['results']))
    
    if debug:
        display(df)
        print(results)
        
    # branch 1: if there is only one type of result:
    if len(set(df['results']))==1:
        if debug: print('In Branch 1!')
        d = process_single_type(d, df)
    
    # other wise there are at least 2 types
    # branch 2: if there are conflicting results
    elif (('Pass' in results) or ('Pass w/ Conditions' in results)) and ('Fail' in results):
        if debug: print('In Branch 2!')
        d = {}
        
    # now we know there are no conflicting results
    # branch 3: if we have both 'Pass' and Pass w/ Conditions'
    elif ('Pass' in results) and ('Pass w/ Conditions' in results):
        if debug: print('In Branch 3!')
        df = df[df['results'].isin(['Pass', 'Pass w/ Conditions'])]
        d['results'] = 'Pass w/ Conditions'
        d['inspection_id'] = df['inspection_id'].values[0] + '-s'
        d['violations'] = np.nan
        if (~df['violations'].isnull()).sum() > 0:
            d['violations'] = ' | '.join([violation for violation in list(set(df['violations'])) if not pd.isna(violation)])
        
    # branch 4: if at least one conclusive result 
    elif ('Pass' in results) or ('Pass w/ Conditions' in results) or ('Fail' in results):
        if debug: print('In Branch 4!')
        df = df[df['results'].isin(conclusive_results)]
        d = process_single_type(d, df)
        
    # all from non-conclusive results
    # branch 5
    elif ('Not Ready' in results):
        if debug: print('In Branch 5!')
        d = choose_non_conclusive_type(d, df, 'Not Ready')      
        
    # branch 6
    elif ('Out of Business' in results):
        if debug: print('In Branch 6!')
        d = choose_non_conclusive_type(d, df, 'Out of Business')  
        
    # branch 7
    elif ('Business Not Located' in results):     
        if debug: print('In Branch 7!')
        d = choose_non_conclusive_type(d, df, 'Business Not Located')    

    return pd.Series(d, index=['inspection_id', 'results', 'violations'])
    
inspections_duplicated_agg = inspections_duplicated.groupby(by=['facility_id', 'inspection_date', 'inspection_type'], dropna=False).apply(inspection_agg).reset_index()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


Now we need to remove the records with null information (for example when there are conflicting results):

In [94]:
inspections_duplicated_agg = inspections_duplicated_agg[~inspections_duplicated_agg['inspection_id'].isnull()]

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#    display(inspections_duplicated_agg)

Next, we need to process the whole dataset `inspection_results`, to replace the records in `inspections_duplicated`, with the aggregated results from `inspections_duplicated_agg`.

In [95]:
inspection_results_dedup = pd.concat([
    inspection_results[~inspection_results['inspection_id'].isin(list(inspections_duplicated['inspection_id']))], 
    inspections_duplicated_agg
    ], ignore_index=True)
inspection_results_dedup

,inspection_id,inspection_date,inspection_type,results,violations,facility_id
0,2079132,2017-08-28,license,Pass,NaN,19402
1,2078553,2017-08-15,license,Not Ready,NaN,19402
2,2079129,2017-08-28,license,Not Ready,NaN,15093
3,2079125,2017-08-28,license re-inspection,Not Ready,NaN,25450
4,2072040,2017-08-08,license,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,25450
...,...,...,...,...,...,...
153312,1949159-s,2016-08-11,canvass,Pass,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",33768
153313,1080244-s,2012-04-16,canvass,Pass,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,33835
153314,1567147-s,2015-09-03,canvass,Pass w/ Conditions,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA...",33866
153315,78338-s,2010-03-09,canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,33898


Confirm the change:

In [96]:
inspection_dup_counts = inspection_results_dedup.groupby(by=['facility_id', 'inspection_date', 'inspection_type'], dropna=False).size().rename('count').reset_index()\
    .sort_values(by='count', ascending=False)
inspection_dup_counts[inspection_dup_counts['count'] > 1]

,facility_id,inspection_date,inspection_type,count


Example of collapsed record:

In [97]:
temp = inspection_results_dedup[(inspection_results_dedup['facility_id']==27465) & (inspection_results_dedup['inspection_date']=='2010-06-07')]
temp

,inspection_id,inspection_date,inspection_type,results,violations,facility_id
153247,160390-s,2010-06-07,canvass,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",27465


In [98]:
print(temp['violations'].values[0])

35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: The walls and ceilings shall be in good repair and easily cleaned. 
 
DROP CEILING PANEL OVER DEEP FRYER IN POOR REPAIR. MUST REPLACE.   | 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: All food and non-food contact equipment and utensils shall be smooth, easily cleanable, and durable, and shall be in good repair. 
 
MUST REMOVE ICE BUILD UP FROM INTERIOR OF REACH IN FREEZER.   | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: The floors shall be constructed per code, be smooth and easily cleaned, and be kept clean and in good repair. 
STANDING WATER NEXT TO GREASE TRAP. INSTRUCTED TO CLEAN AND MAINTAIN. | 35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: 

In [99]:
inspections = inspection_results_dedup[['inspection_id', 'inspection_date', 'inspection_type', 'results']].sort_values(by='inspection_date').reset_index(drop=True)
inspections.to_csv("data/tables/inspections.csv", index=False)
inspections

,inspection_id,inspection_date,inspection_type,results
0,67732,2010-01-04,license re-inspection,Pass
1,67733,2010-01-04,license re-inspection,Pass
2,52234,2010-01-04,license re-inspection,Pass
3,67757,2010-01-04,tag removal,Pass
4,70269,2010-01-04,license re-inspection,Pass
...,...,...,...,...
153312,2079123,2017-08-28,license,Pass
153313,2079125,2017-08-28,license re-inspection,Not Ready
153314,2079129,2017-08-28,license,Not Ready
153315,2079105,2017-08-28,license re-inspection,Pass


In [100]:
facility_inspection = inspection_results_dedup[['facility_id', 'inspection_id']].sort_values(by='facility_id').reset_index(drop=True)
facility_inspection.to_csv("data/tables/facility_inspection.csv", index=False)
facility_inspection

,facility_id,inspection_id
0,0,467259
1,0,1285263
2,1,1296904
3,1,1395782
4,1,277104
...,...,...
153312,34034,1376090
153313,34034,1099029
153314,34034,1098968
153315,34035,585594


## 3.4 Violation and  Inspection_Violation tables

In [101]:
violation_df = inspection_results_dedup[~pd.isnull(inspection_results_dedup['violations'])][['inspection_id', 'violations']].reset_index(drop=True)
violation_df

,inspection_id,violations
0,2072040,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
1,2079123,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
2,2079105,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
3,2078632,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...
4,2079104,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...
...,...,...
122502,1949159-s,"16. FOOD PROTECTED DURING STORAGE, PREPARATION..."
122503,1080244-s,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...
122504,1567147-s,"6. HANDS WASHED AND CLEANED, GOOD HYGIENIC PRA..."
122505,78338-s,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...


Still use this example to figure out how to process the violations:

In [102]:
print(violation_df[violation_df['inspection_id']=='160390-s']['violations'].values[0])

35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: The walls and ceilings shall be in good repair and easily cleaned. 
 
DROP CEILING PANEL OVER DEEP FRYER IN POOR REPAIR. MUST REPLACE.   | 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: All food and non-food contact equipment and utensils shall be smooth, easily cleanable, and durable, and shall be in good repair. 
 
MUST REMOVE ICE BUILD UP FROM INTERIOR OF REACH IN FREEZER.   | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: The floors shall be constructed per code, be smooth and easily cleaned, and be kept clean and in good repair. 
STANDING WATER NEXT TO GREASE TRAP. INSTRUCTED TO CLEAN AND MAINTAIN. | 35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: 

**[solution]**  
- first, split violations using " | ", and expand the records (so one ispection_id can now have multiple rows, one row for one violation record).  
- then, for each violation record, we seperate violation code, description, and comment.  

In [103]:
violation_df = violation_df.assign(
    violations=violation_df['violations'].str.split(' \| ')).explode('violations').reset_index(drop=True)
violation_df.head(10)

,inspection_id,violations
0,2072040,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
1,2072040,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D..."
2,2072040,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...
3,2079123,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
4,2079123,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO..."
5,2079105,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
6,2079105,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...
7,2079105,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO..."
8,2079105,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...
9,2079105,"41. PREMISES MAINTAINED FREE OF LITTER, UNNECE..."


Example:

In [104]:
violation_df[violation_df['inspection_id']=='160390-s']

,inspection_id,violations
566345,160390-s,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
566346,160390-s,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...
566347,160390-s,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO..."
566348,160390-s,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR..."
566349,160390-s,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO..."
566350,160390-s,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...
566351,160390-s,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...


There may exist multiple violations for the same inspection_id and violation code, but with different comments:

In [105]:
violation_df.loc[566348, 'violations']

'35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: The walls and ceilings shall be in good repair and easily cleaned. \nPEELING PAINT NOTED ON WALL NEXT TO DEEP FRYER, INSTRUCTED MANAGER TO REMOVE AND MAINTAIN.'

In [106]:
violation_df.loc[566349, 'violations']

'34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: The floors shall be constructed per code, be smooth and easily cleaned, and be kept clean and in good repair. \n \nMUST DETAIL CLEAN FLOORS UNDER GRILL AND REMOVE FOOD DEBRIS AND GREASE BUILD UP.  '

In [107]:
def split_violation(violation):
    violation, comment = violation.split(' - Comments: ')
    code, description = violation.split('. ', 1)
    return code, description, comment
violation_df['code'], violation_df['description'], violation_df['comment'] = zip(*violation_df['violations'].apply(split_violation))
violation_df = violation_df.drop(columns='violations').drop_duplicates().reset_index(drop=True)
violation_df

,inspection_id,code,description,comment
0,2072040,18,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...,NO LICENSE PEST CONTROL LOG BOOK AT THIS TIME ...
1,2072040,11,"ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, DESIG...","NO EXPOSED HAND SINK FOR REAR SERVICE AREA, IN..."
2,2072040,8,SANITIZING RINSE FOR EQUIPMENT AND UTENSILS: ...,"NO DISH WASHING FACILITIES ON SITE, (NO THREE ..."
3,2079123,35,"WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTE...",MUST PROVIDE OVERHEAD PROTECTION FOR COFFEE/TE...
4,2079123,34,"FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD RE...",FLOORS AT COFFEE/TEA STATIONS MUST BE MADE SMO...
...,...,...,...,...
566561,58461-s,32,FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DE...,All food and non-food contact equipment and ut...
566562,58461-s,34,"FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD RE...","The floors shall be constructed per code, be s..."
566563,58461-s,38,VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQ...,"Ventilation: All plumbing fixtures, such as to..."
566564,58461-s,41,"PREMISES MAINTAINED FREE OF LITTER, UNNECESSAR...",All parts of the food establishment and all pa...


In [108]:
violations = violation_df[['code', 'description']].drop_duplicates()\
    .sort_values(by='code', key=lambda val: val.astype(int)).reset_index(drop=True)
violations.to_csv("data/tables/violations.csv", index=False)
violations

,code,description
0,1,"SOURCE SOUND CONDITION, NO SPOILAGE, FOODS PRO..."
1,2,FACILITIES TO MAINTAIN PROPER TEMPERATURE
2,3,POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATURE R...
3,4,SOURCE OF CROSS CONTAMINATION CONTROLLED I.E. ...
4,5,PERSONNEL WITH INFECTIONS RESTRICTED: NO OPEN ...
5,6,"HANDS WASHED AND CLEANED, GOOD HYGIENIC PRACTI..."
6,7,WASH AND RINSE WATER: CLEAN AND PROPER TEMPERA...
7,8,SANITIZING RINSE FOR EQUIPMENT AND UTENSILS: ...
8,9,"WATER SOURCE: SAFE, HOT & COLD UNDER CITY PRES..."
9,10,"SEWAGE AND WASTE WATER DISPOSAL, NO BACK SIPHO..."


In [109]:
def custom_sorting(x):
    if x.name == 'inspection_id':
        return x.str.replace('-s', '').astype(int)    
    elif x.name == 'code':
        return x.astype(int)
    else:
        return x

inspection_violation = violation_df[['inspection_id', 'code', 'comment']]\
    .sort_values(by=['inspection_id', 'code'], key=custom_sorting, ignore_index=True)\
    .rename(columns={'code': 'violation_code', 'comment': 'violation_comment'})
inspection_violation.to_csv("data/tables/inspection_violation.csv", index=False)
inspection_violation

,inspection_id,violation_code,violation_comment
0,44247,30,All food not stored in the original container ...
1,44247,32,OBSERVED TORN DOOR GASKET ON DOOR OF 'CHILL IS...
2,44247,33,All food and non-food contact surfaces of equi...
3,44247,34,"The floors shall be constructed per code, be s..."
4,44247,37,Toilet rooms shall be completely enclosed and ...
...,...,...,...
566561,2079219,31,MUST INVERT MULTIUSE UTENSILS(POTS AND PANS) O...
566562,2079219,32,MUST ELIMINATE GROCERY BAGS USED FOR FOOD STOR...
566563,2079219,35,MUST CLEAN DUST BUILD UP FROM VENTS IN DINING ...
566564,2079219,36,MUST PROVIDE SHATTER PROOF BULBS OR LIGHT SHIE...


Again, note that a certain inspection_id/violation code combination can have multiple comments. For example:

In [110]:
inspection_violation[inspection_violation['inspection_id']=='160390-s']

,inspection_id,violation_code,violation_comment
23648,160390-s,32,All food and non-food contact equipment and ut...
23649,160390-s,32,All food and non-food contact equipment and ut...
23650,160390-s,33,All food and non-food contact surfaces of equi...
23651,160390-s,34,"The floors shall be constructed per code, be s..."
23652,160390-s,34,"The floors shall be constructed per code, be s..."
23653,160390-s,35,The walls and ceilings shall be in good repair...
23654,160390-s,35,The walls and ceilings shall be in good repair...
